In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import keras

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model_h as modellib
from mrcnn import visualize
from mrcnn.model_h import log
sys.path.append(os.path.join(ROOT_DIR, "samples/interior/"))  # To find local version
import interior_seq

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Path to Shapes trained weights
SHAPES_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_shapes.h5")

/home/erbachj/MaskRCNN/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/erbachj/MaskRCNN/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/erbachj/MaskRCNN/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/erbachj/MaskRCNN/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
# Shapes toy dataset
# import shapes
# config = shapes.ShapesConfig()

# MS COCO Dataset
class InferenceConfig(interior_seq.InteriorNetConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    TOP_DOWN_PYRAMID_SIZE = 64
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 23  # background + num classes
    vmin = -1.07
    vmax = 1.07
    nvox = 40
    nvox_z = 40
    vsize = float(vmax - vmin) / nvox
    vox_bs = 1
    im_bs = 1
    samples = 10
    NUM_VIEWS = 2
    RECURRENT = False
    USE_RPN_ROIS = True
    LEARNING_RATE = 0.001
    GRID_REAS = 'ident'
    BACKBONE = 'resnet50'
    VANILLA = False
            
config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
GRID_REAS                      ident
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  640
IMAGE_META_SIZE                35
IMAGE_MIN_DIM                  640
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [640 640   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE         

In [3]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [9]:
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=MODEL_DIR)

shape_norm_boxes_graph: Tensor("lambda_59/strided_slice:0", shape=(2,), dtype=int32)
shape_norm_boxes_graph: Tensor("lambda_59/strided_slice_1:0", shape=(2,), dtype=int32)
shape of input: [2, 640, 640, 3]
input_image_shape: [None, 2, 640, 640, 3]
fn image_shape: [None, 2, 640, 640, 3]
shape_out: [None, 2, 160, 160, 64]
shape_out: [None, 2, 80, 80, 64]
shape_out: [None, 2, 40, 40, 64]
shape_out: [None, 2, 20, 20, 64]
shape_out: [None, 2, 10, 10, 64]
finished whole bb model
input_image_shape: [None, 2, 640, 640, 3]
fn image_shape: [None, 2, 640, 640, 3]
shape_out: [None, 2, 160, 160, 64]
shape_out: [None, 2, 80, 80, 64]
shape_out: [None, 2, 40, 40, 64]
shape_out: [None, 2, 20, 20, 64]
shape_out: [None, 2, 10, 10, 64]
P2_shape: [None, 2, 160, 160, 64]
Tensor("unproj_P2_2/Tile:0", shape=(?,), dtype=int32)
shape of Kmat: [None, None, 3, 3]
shape of Rcam: [None, 3, 4]
grid position shape: [1, 3]
grid shape: [4, 64000]
Tensor("unproj_P2_2/Tile_3:0", shape=(?,), dtype=int32)
shape of Kmat: [No

In [15]:
model_path = os.path.join(ROOT_DIR, 'logs/interiornet20200702T1647/mask_rcnn_interiornet_0003.h5')
from keras.engine import saving
import h5py
# f = h5py.File(os.path.join(model_path), mode='r')
# folder, weight_name = os.path.split(model_path)
# epoch = weight_name[-7:-3]
# model_path_bb = folder+'/backbone_callb_epoch_{}.h5'.format(epoch)
# f = h5py.File(os.path.join(model_path_bb), mode='r')
# #f = h5py.File(COCO_MODEL_PATH, mode='r')
# for layer in model.keras_model.layers:
#     if layer.name == 'backbone':
#         layers = layer.layers
#         print(layer.__class__.__name__)
#         saving.load_weights_from_hdf5_group_by_name(f, layer.layers)
#         break

print(model_path)
model.load_weights(model_path, by_name=True)

/media/scratch1/erbachj/Mask_RCNN/logs/interiornet20200702T1647/mask_rcnn_interiornet_0003.h5
normal model
input_image
time_distributed_31
conv1
bn_conv1
activation_315
time_distributed_32
res2a_branch2a
bn2a_branch2a
activation_316
res2a_branch2b
bn2a_branch2b
activation_317
res2a_branch2c
res2a_branch1
bn2a_branch2c
bn2a_branch1
add_106
activation_318
res2b_branch2a
bn2b_branch2a
activation_319
res2b_branch2b
bn2b_branch2b
activation_320
res2b_branch2c
bn2b_branch2c
add_107
activation_321
res2c_branch2a
bn2c_branch2a
activation_322
res2c_branch2b
bn2c_branch2b
activation_323
res2c_branch2c
bn2c_branch2c
add_108
activation_324
res3a_branch2a
bn3a_branch2a
activation_325
res3a_branch2b
bn3a_branch2b
activation_326
res3a_branch2c
res3a_branch1
bn3a_branch2c
bn3a_branch1
add_109
activation_327
res3b_branch2a
bn3b_branch2a
activation_328
res3b_branch2b
bn3b_branch2b
activation_329
res3b_branch2c
bn3b_branch2c
add_110
activation_330
res3c_branch2a
bn3c_branch2a
activation_331
res3c_branch2

In [16]:
visualize.display_weight_stats(model)

WEIGHT NAME,SHAPE,MIN,MAX,STD
conv1_5/kernel:0,"(7, 7, 3, 64)",-0.0427,+0.0427,+0.0246
conv1_5/bias:0,"(64,)",+0.0000,+0.0000,+0.0000
bn_conv1_5/gamma:0*** dead?,"(64,)",+1.0000,+1.0000,+0.0000
bn_conv1_5/beta:0*** dead?,"(64,)",+0.0000,+0.0000,+0.0000
bn_conv1_5/moving_mean:0*** dead?,"(64,)",+0.0000,+0.0000,+0.0000
bn_conv1_5/moving_variance:0*** dead?,"(64,)",+1.0000,+1.0000,+0.0000
res2a_branch2a_5/kernel:0,"(1, 1, 64, 64)",-0.2164,+0.2165,+0.1241
res2a_branch2a_5/bias:0,"(64,)",+0.0000,+0.0000,+0.0000
bn2a_branch2a_5/gamma:0*** dead?,"(64,)",+1.0000,+1.0000,+0.0000
bn2a_branch2a_5/beta:0*** dead?,"(64,)",+0.0000,+0.0000,+0.0000


In [ ]:
for layer in model.keras_model.layers:
    print(layer.weights)
